### Temp - Will be convereted into main.py once complete
Currently being used for testing

In [1]:
import torch
import pickle
import os
import numpy as np

from data.process_data import load_data, augment_data, preprocess_data, to_tensors, split_batch
from models.quickdraw_cnn import QuickDrawCNN_V1, QuickDrawCNN_V2
from training.model_train import train

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
image_dir = "data/numpy_bitmap/"

Device: cuda


In [3]:
categories = os.listdir(image_dir)
p1, p2 = categories[:len(categories)//2], categories[len(categories)//2:]

In [4]:
features_p1, labels_p1 = load_data(p1, image_dir, file_standardize=False, sample=3)
features_p2, labels_p2 = load_data(p2, image_dir, file_standardize=False, sample=3)

Loading data:  20%|█▉        | 34/172 [00:03<00:12, 10.91it/s] 


KeyboardInterrupt: 

In [17]:
# Preprocessing data split array -> 28x28 numpy
features_p1, labels_p1 = preprocess_data(features_p1, labels_p1)
features_p2, labels_p2 = preprocess_data(features_p2, labels_p2)

Preprocessing 172 images...


Processing labels: 100%|██████████| 172/172 [00:00<00:00, 19804.00it/s]


Preprocessing 173 images...


Processing labels: 100%|██████████| 173/173 [00:00<00:00, 110950.24it/s]


In [18]:
labels = labels_p1 + labels_p2
label_map = {label: i for i, label in enumerate(set(labels))}

In [ ]:
def to_tensors(features: np.array, labels: np.array, labels_map: dict, device: str = 'cpu'):
    
    '''
    Normalize objects convert the features and labels into tensor objects
    
    args:
    - features: numpy array with the features
    - labels: numpy array with the labels
    - labels_map: dictionary with the labels mapping
    - device: device to use
    
    returns: features, labels, labels_map
    '''
    
    print(f"Converting {len(labels)} images to tensors, using {device}...")
        
    labels = [labels_map[label] for label in labels] # convert labels to integers
    
    features = torch.tensor(features).to(device) / 255.0 # normalizing 0-255 -> 0-1
    features = features.unsqueeze(1) # adding channel dimension for CNN, 1 channel for grayscale
    labels = torch.tensor(labels).to(device)
    
    print(f"Features shape: {features.shape}, Labels shape: {labels.shape}")
    
    return features, labels

features_p1, labels_p1 = to_tensors(features_p1, labels_p1, label_map, device)
features_p2, labels_p2 = to_tensors(features_p2, labels_p2, label_map, device)

Converting 172 images to tensors, using cuda...


OutOfMemoryError: CUDA out of memory. Tried to allocate 73.70 GiB. GPU 0 has a total capacity of 11.99 GiB of which 0 bytes is free. Of the allocated memory 18.43 GiB is allocated by PyTorch, and 0 bytes is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [41]:
# Split further for memory
features_p1_1, features_p1_2 = features_p1[:len(features_p1) // 2], features_p1[len(features_p1) // 2:]
labels_p1_1, labels_p1_2 = labels_p1[:len(labels_p1) // 2], labels_p1[len(labels_p1) // 2:]
features_p2_1, features_p2_2 = features_p2[:len(features_p2) // 2], features_p2[len(features_p2) // 2:]
labels_p2_1, labels_p2_2 = labels_p2[:len(labels_p2) // 2], labels_p2[len(labels_p2) // 2:]

# Augment data
features_p1_1, labels_p1_1 = augment_data(features_p1_1, labels_p1_1, rot=0, h_flip=True, v_flip=False)
features_p1_2, labels_p1_2 = augment_data(features_p1_2, labels_p1_2, rot=0, h_flip=True, v_flip=False)
features_p2_1, labels_p2_1 = augment_data(features_p2_1, labels_p2_1, rot=0, h_flip=True, v_flip=False)
features_p2_2, labels_p2_2 = augment_data(features_p2_2, labels_p2_2, rot=0, h_flip=True, v_flip=False)

Starting size: (12617452, 28, 28)
Flipping images horizontally...


Flipping images horizontally...: 100%|██████████| 12617452/12617452 [00:06<00:00, 2073612.49it/s]


Augmented size: (25234904, 28, 28)
Starting size: (12617453, 28, 28)
Flipping images horizontally...


Flipping images horizontally...: 100%|██████████| 12617453/12617453 [00:06<00:00, 2093258.78it/s]


MemoryError: Unable to allocate 18.4 GiB for an array with shape (25234906, 28, 28) and data type uint8

In [15]:
# Combine features and labels
features = np.concatenate((features_p1_1, features_p1_2, features_p2_1, features_p2_2), axis=0)
labels = np.concatenate((labels_p1_1, labels_p1_2, labels_p2_1, labels_p2_2), axis=0)

In [11]:
# Split into train, test, and validation sets
train_loader, test_loader, val_loader = split_batch(features, labels, batch_size=64)

TypeError: split_batch() got multiple values for argument 'batch_size'

In [ ]:
model = QuickDrawCNN_V2().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_loss, val_loss, train_acc, val_acc = train(model = model,
                                                  train_loader = train_loader,
                                                  val_loader = val_loader,
                                                  epochs = 10,
                                                  criterion = criterion,
                                                  optimizer = optimizer,
                                                  device = device)   

In [ ]:
save_path = "models/quickdraw_cnn_v2.pth"
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}.")

In [ ]:
with open("models/labels_map.pkl", "wb") as f:
    pickle.dump(labels_map, f)
    pickle.dump(train_loss, f)
    pickle.dump(val_loss, f)
    pickle.dump(train_acc, f)
    pickle.dump(val_acc, f)